In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
import cv2
import random
import json, os
from pycocotools.coco import COCO
from skimage import io
from matplotlib import pyplot as plt

In [ ]:
import detectron2
from pathlib import Path
import random, cv2, os
import matplotlib.pyplot as plt
import numpy as np
import pycocotools.mask as mask_util
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.evaluation.evaluator import DatasetEvaluator
setup_logger()

## Visualize and train/valid Split

In [ ]:
train_dataset_dir="../input/cow-seg/train_dataset/200/"
train_label="../input/cow-seg/train_dataset/200/data.json"

#copy from https://blog.csdn.net/wtandyn/article/details/109751015

def visualization_bbox_seg(num_image, json_path, img_path, *str):# 需要画图的是第num副图片， 对应的json路径和图片路径

    coco = COCO(json_path)

    if len(str) == 0:
        catIds = []
    else:
        catIds = coco.getCatIds(catNms = [str[0]])  # 获取给定类别对应的id 的dict（单个内嵌字典的类别[{}]）
        catIds = coco.loadCats(catIds)[0]['id'] # 获取给定类别对应的id 的dict中的具体id

    list_imgIds = coco.getImgIds(catIds=catIds ) # 获取含有该给定类别的所有图片的id
    img = coco.loadImgs(list_imgIds[num_image-1])[0]  # 获取满足上述要求，并给定显示第num幅image对应的dict
    image = io.imread(img_path + img['file_name'])  # 读取图像
    image_name =  img['file_name'].split('/')[-1] # 读取图像名字
    image_id = img['id'] # 读取图像id

    img_annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None) # 读取这张图片的所有seg_id
    img_anns = coco.loadAnns(img_annIds)

    for i in range(len(img_annIds)):
        x, y, w, h = img_anns[i-1]['bbox']  # 读取边框
        image = cv2.rectangle(image, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 255), 2)

    plt.rcParams['figure.figsize'] = (20.0, 20.0)
    plt.imshow(image)
    coco.showAnns(img_anns)
    plt.show()

visualization_bbox_seg(1, train_label, train_dataset_dir,'cow') # 最后一个参数不写就是画出一张图中的所有类别
with open(train_label,'r') as f:
    data=json.load(f)
#
train_dict={
    'images':[],
    'categories':[],
    'annotations':[]
}
val_dict={
    'images':[],
    'categories':[],
    'annotations':[]
}
train_dict['categories']=data['categories']
val_dict['categories']=data['categories']
images=data['images']
annos=data['annotations']
#20张用于验证
train_img_lst=[]
train_ann_lst=[]
val_img_lst=[]
val_ann_lst=[]
for i in range(len(images)):
    img_info=images[i]
    img_id=img_info['id']
    #遍历annos，找到对应image_id的所有标注
    tmp=[]
    for ann in annos:
            if ann['image_id']==img_id:
                tmp.append(ann)
    #
    if i%10==0:
        val_img_lst.append(img_info)
        val_ann_lst+=tmp
    else:
        train_img_lst.append(img_info)
        train_ann_lst+=tmp
#
train_dict['images']=train_img_lst
val_dict['images']=val_img_lst
train_dict['annotations']=train_ann_lst
val_dict['annotations']=val_ann_lst
print("train: {},valid: {}".format(len(train_img_lst),len(val_img_lst)))
#
with open('val.json','w') as f:
    tmp=json.dumps(val_dict)
    f.write(tmp)
with open('train.json','w') as f:
    tmp=json.dumps(train_dict)
    f.write(tmp)

In [ ]:
dataDir=Path('../input/cow-seg/train_dataset/200/')
cfg = get_cfg()
cfg.INPUT.MASK_FORMAT='bitmask'
register_coco_instances('cow_train',{}, 'train.json', dataDir)
register_coco_instances('cow_val',{},'val.json', dataDir)
metadata = MetadataCatalog.get('cow_train')
train_ds = DatasetCatalog.get('cow_train')

In [ ]:
d = train_ds[42]
img = cv2.imread(d["file_name"])
visualizer = Visualizer(img[:, :, ::-1], metadata=metadata)
out = visualizer.draw_dataset_dict(d)
plt.figure(figsize = (20,15))
plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:

import logging
import os
from collections import OrderedDict
import torch

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
)
from detectron2.modeling import GeneralizedRCNNWithTTA


def build_evaluator(cfg, dataset_name, output_folder=None):
    """
    Create evaluator(s) for a given dataset.
    This uses the special metadata "evaluator_type" associated with each builtin dataset.
    For your own dataset, you can simply create an evaluator manually in your
    script and do not have to worry about the hacky if-else logic here.
    """
    if output_folder is None:
        output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
    evaluator_list = []
    evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
    if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
        evaluator_list.append(
            SemSegEvaluator(
                dataset_name,
                distributed=True,
                output_dir=output_folder,
            )
        )
    if evaluator_type in ["coco", "coco_panoptic_seg"]:
        evaluator_list.append(COCOEvaluator(dataset_name, output_dir=output_folder))
    if evaluator_type == "coco_panoptic_seg":
        evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
    if evaluator_type == "cityscapes_instance":
        return CityscapesInstanceEvaluator(dataset_name)
    if evaluator_type == "cityscapes_sem_seg":
        return CityscapesSemSegEvaluator(dataset_name)
    elif evaluator_type == "pascal_voc":
        return PascalVOCDetectionEvaluator(dataset_name)
    elif evaluator_type == "lvis":
        return LVISEvaluator(dataset_name, output_dir=output_folder)
    if len(evaluator_list) == 0:
        raise NotImplementedError(
            "no Evaluator for the dataset {} with the type {}".format(dataset_name, evaluator_type)
        )
    elif len(evaluator_list) == 1:
        return evaluator_list[0]
    return DatasetEvaluators(evaluator_list)


class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains pre-defined default logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can write your
    own training loop. You can use "tools/plain_train_net.py" as an example.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        return build_evaluator(cfg, dataset_name, output_folder)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        logger = logging.getLogger("detectron2.trainer")
        # In the end of training, run an evaluation with TTA
        # Only support some R-CNN models.
        logger.info("Running inference with test-time augmentation ...")
        model = GeneralizedRCNNWithTTA(cfg, model)
        evaluators = [
            cls.build_evaluator(
                cfg, name, output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
            )
            for name in cfg.DATASETS.TEST
        ]
        res = cls.test(cfg, model, evaluators)
        res = OrderedDict({k + "_TTA": v for k, v in res.items()})
        return res


## training

In [ ]:
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("cow_train",)
cfg.DATASETS.TEST = ("cow_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.0005 
cfg.SOLVER.MAX_ITER = 2000    
cfg.SOLVER.STEPS = []        
#cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = .5
cfg.TEST.EVAL_PERIOD = 4*len(DatasetCatalog.get('cow_train')) // cfg.SOLVER.IMS_PER_BATCH  # Once per epoch


In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## Infer Visualize

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
#cfg.MODEL.WEIGHTS = '../input/cow-seg-with-detectron-training/output/model_final.pth'
predictor = DefaultPredictor(cfg)
dataset_dicts = DatasetCatalog.get('cow_val')
outs = []
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata = MetadataCatalog.get('cow_train'), 
                    
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out_pred = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    visualizer = Visualizer(im[:, :, ::-1], metadata=MetadataCatalog.get('cow_train'))
    out_target = visualizer.draw_dataset_dict(d)
    outs.append(out_pred)
    outs.append(out_target)
_,axs = plt.subplots(len(outs)//2,2,figsize=(40,45))
for ax, out in zip(axs.reshape(-1), outs):
    ax.imshow(out.get_image()[:, :, ::-1])


## get the submit.json

In [ ]:

#
from tqdm import tqdm
import pycocotools.mask as mask_util
pre_sub_json=[]
dataDir='../input/cow-seg/test_dataset_A/images'
ids, masks=[],[]
id_inx=0
image_id_inx=0
test_names = os.listdir(dataDir)
for name in tqdm(test_names):
    fn=os.path.join(dataDir,name)
    im = cv2.imread(fn)
    pred = predictor(im)
    image_id_inx+=1
    take = pred['instances'].scores >= 0.05
    instances=pred['instances']
    pred_masks_rle = [mask_util.encode(np.asfortranarray(mask)) for mask in instances.pred_masks[take].cpu()]
    pred_masks = pred_masks_rle
    pred_bboxs = instances.pred_boxes.tensor[take]
    pred_scores = instances.scores[take]
    pred_bboxs = pred_bboxs.cpu().numpy()
    pred_scores = pred_scores.cpu().numpy()
    for i in range(len(pred_masks)):
        id_inx+=1
        pre_sub={
              'image_id': '',
              'category_id': 1,
              'segmentation': {},
              'score': 1.
        }
        pre_sub['image_id']='images/'+name
        mask=pred_masks[i]
        counts=mask['counts']
        #mask['counts']=mask['counts'].decode('utf-8')
        counts=str(counts, encoding='utf-8')
        mask['counts']=counts
        bbox=pred_bboxs[i]
        score=pred_scores[i]
        pre_sub['segmentation']=mask
        pre_sub['score']=float(score)
        #x1=bbox[0]
        #y1=bbox[1]
        #w=bbox[2]-bbox[0]
        #h=bbox[3]-bbox[1]
        #pre_sub['bbox']=[int(bbox[0]),int(bbox[1]),int(w),int(h)]
        ##deal with segmentation
        #img=mask.astype(np.uint8)
        #img[img==1]=255
        #edge=cv2.Canny(img,30,100)
        #xs,ys=np.where(edge==255)
        #polygon=[]
        #for j in range(0,len(xs),10):
        #    polygon.append(int(xs[j]))
        #    polygon.append(int(ys[j]))
        #pre_sub['segmentation']=[polygon]
        #pre_sub['area']=int(w*h)
        #print(len(polygon))
        pre_sub_json.append(pre_sub)
    #break
len(pre_sub_json)

In [ ]:
with open('submission.json','w') as f:
    data=json.dumps(pre_sub_json)
    f.write(data)